In [ ]:
import import_ipynb

from keras.layers import Input, Dense, Concatenate, GlobalAveragePooling1D, Dropout
from IPython.display import display, Image, update_display, HTML
from tensorflow.keras.utils import plot_model
from keras.models import Model, Sequential
from transformer import TransformerBlock
from keras.regularizers import L1L2
from threading import Thread, Lock
from dataclasses import dataclass
import user_ad_interaction
import keras_tuner as kt
import tensorflow as tf
from queue import Queue
import adgorithm_htuner
from tqdm import tqdm
import pandas as pd
import numpy as np
import bisect
import tools
import users
import os

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm


In [ ]:
root_directories = ["ADS16_Benchmark_part1", "ADS16_Benchmark_part2"]

In [ ]:
tuner = tools.HyperbandWithBatchSize(
    user_ad_interaction.create_model,
    objective=kt.Objective("val_rating_mae", direction="min"),
    max_epochs=100,
    factor=3,
    hyperband_iterations=5,
    directory="user_ad_interaction_model_logs/hypertraining",
    project_name="user_ad_interaction"
)
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

user_model = user_ad_interaction.create_model(best_hps)
user_model.load_weights("user_ad_interaction_model_logs/checkpoints/model-000020-1.202267.hdf5")

In [ ]:
ad_ftrs, num_categories, real_user_ftrs, _ = user_ad_interaction.load_user_and_ad_ftrs(root_directories)
user_pca_normal_params = users.approximate_normal_params(real_user_ftrs.T)

In [ ]:
rating_to_exp_ctr = {
    1: 0.014925,
    2: 0.024786,
    3: 0.031071,
    4: 0.040562,
    5: 0.068341
}

In [ ]:
tuner = tools.HyperbandWithBatchSize(
    lambda hp: RLModel(hp).model,
    objective=kt.Objective("val_output_1_loss", direction="min"),
    max_epochs=100,
    factor=3,
    hyperband_iterations=5,
    directory="adgorithm_logs/hypertraining",
    project_name="adgorithm_htuning"
)
rl_best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

model = adgorithm_htuner.create_rl_model_for_htuning(rl_best_hps)
model.summary()
c = tools.ModelDisplayer()
c.model = model
c.on_epoch_end(0)

In [ ]:
@dataclass
class RankedAd:
    ad_id: int
    value: float

    def __lt__(self, other):
        return self.value < other.value
    
    def __repr__(self):
        return f"{self.ad_id}{{{self.value}}}"

class Memory:
    def __init__(self, size=100):
        self.size = size
        self.ad_ftrs = (
            np.zeros((self.size, *ad_ftrs[0].shape[1:])),
            np.zeros((self.size, *ad_ftrs[1].shape[1:]))
        )
        self.rewards = np.zeros((self.size))
        self.ptr = 0
        self.full = False
    
    def store(self, ad_ftrs, reward):
        self.ad_ftrs[0][self.ptr] = ad_ftrs[0]
        self.ad_ftrs[1][self.ptr] = ad_ftrs[1]
        self.rewards[self.ptr] = reward

        self.ptr += 1
        if self.ptr >= self.size:
            self.full = True
            self.ptr = 0
    
    def get_training_data(self):
        if self.full:
            return (self.ad_ftrs, self.rewards)
        else:
            return ((self.ad_ftrs[0][:self.ptr], self.ad_ftrs[1][:self.ptr]), self.rewards[:self.ptr])

class User:
    def __init__(self, ftrs=None):
        self.ftrs = ftrs
        if self.ftrs is None:
            self.ftrs = users.generate_synthetic_pca_ftrs(1, user_pca_normal_params)[0]
        self.rl_model = adgorithm_htuner.create_rl_model_for_htuning(rl_best_hps)
        self.sorted_ads = []
        self.memory = Memory()
        self.clicks = {}
        self.diminishing_returns_coeff = 0.9
        self.sum_exp_reward = 0
        self.ad_memory_size = 100
    
    def get_true_exp_ctr(self, ad_id, ad_ftrs):
        rating = np.argmax(user_model((np.concatenate([self.ftrs, ad_ftrs[0]])[np.newaxis], ad_ftrs[1][np.newaxis]), training=False)) + 1
        exp_ctr = rating_to_exp_ctr[rating]
        if ad_id in self.clicks:
            exp_ctr *= self.diminishing_returns_coeff ** self.clicks[ad_id]
        return exp_ctr

    def predict_reward(self, ad_id, ad_ftrs):
        c = self.clicks[ad_id] if ad_id in self.clicks else 0
        exp_reward = self.rl_model((np.concatenate((ad_ftrs[0], [c]))[np.newaxis], ad_ftrs[1][np.newaxis]))[0,0]
        ra = RankedAd(ad_id, exp_reward)
        self.sum_exp_reward += exp_reward
        for i, existing_ad in enumerate(self.sorted_ads):
            if existing_ad.ad_id == ad_id:
                self.sum_exp_reward -= existing_ad.value
                del self.sorted_ads[i]
                break
        bisect.insort(self.sorted_ads, ra)
    
    def interact_with(self, ad_id, ad_ftrs):
        exp_ctr = self.get_true_exp_ctr(ad_id, ad_ftrs)
        did_click = np.random.uniform() <= exp_ctr
        if did_click:
            if ad_id not in self.clicks:
                self.clicks[ad_id] = 1
            else:
                self.clicks[ad_id] += 1
            AdDatabase.ad_was_clicked(ad_id)
        return did_click
    
    def select_important_ad(self, p_random=0.1):
        if np.random.uniform() <= p_random or len(self.sorted_ads) == 0:
            return AdDatabase.get_random_ad()
        ad_id = self.sorted_ads[np.random.randint(0, len(self.sorted_ads))].ad_id
        return AdDatabase.get_ad(ad_id)
    
    def select_ad(self, p_random=0.1):
        if np.random.uniform() <= p_random or self.sum_exp_reward == 0:
            return AdDatabase.get_random_ad()

        r = np.random.uniform(0, self.sum_exp_reward)
        s = 0
        for ad in self.sorted_ads[::-1]:
            s += ad.value
            if s >= r:
                return AdDatabase.get_ad(ad.ad_id)
        
        assert False
    
    def remember(self, ad_id, ad_ftrs, reward):
        self.memory.store(ad_ftrs, reward)
    
    def learn(self, rl_epochs=10, n_ads_to_fetch=10, p_random=0.1):
        self.rl_model.fit(
            *self.memory.get_training_data(),
            verbose=0,
            epochs=rl_epochs,
            batch_size=32
        )
        for _ in range(n_ads_to_fetch):
            ad = self.select_important_ad(p_random=p_random)
            self.predict_reward(*ad)
        self.sum_exp_reward -= sum(x.value for x in self.sorted_ads[:-self.ad_memory_size])
        self.sorted_ads = self.sorted_ads[-self.ad_memory_size:]


In [ ]:
available_ads = {}
ad_clicks = {}
for i, af0 in enumerate(ad_ftrs[0]):
    af1 = ad_ftrs[1][i]
    available_ads[i] = (af0, af1)
    ad_clicks[i] = 0
ad_id_ctr = len(ad_ftrs[0])
click_thresh = 100

class NullLock:
    def __enter__(self, *args, **kwargs):
        pass
    def __exit__(self, *args, **kwargs):
        pass

class BypassableLock:
    def __init__(self):
        self.lock = Lock()
    
    def __call__(self, use_lock=True):
        return self.lock if use_lock else NullLock()
    
    def __enter__(self, *args, **kwargs):
        return self.lock.__enter__(*args, **kwargs)

    def __exit__(self, *args, **kwargs):
        return self.lock.__exit__(*args, **kwargs)

ad_db_lock = BypassableLock()

class AdDatabase:
    @staticmethod
    def get_ad(ad_id, use_lock=True):
        with ad_db_lock(use_lock):
            if ad_id in available_ads:
                return ad_id, available_ads[ad_id]
            return AdDatabase.get_random_ad(use_lock=False)

    @staticmethod
    def get_random_ad(use_lock=True):
        with ad_db_lock(use_lock):
            ad_id = np.random.choice(list(available_ads.keys()))
            return AdDatabase.get_ad(ad_id, use_lock=False)
    
    @staticmethod
    def ad_was_clicked(ad_id, use_lock=True):
        with ad_db_lock(use_lock):
            ad_clicks[ad_id] += 1
            if ad_clicks[ad_id] >= click_thresh:
                available_ads[ad_id_ctr] = available_ads[ad_id]
                ad_clicks[ad_id_ctr] = 0
                ad_id_ctr += 1
                del ad_clicks[ad_id]
                del available_ads[ad_id]


In [ ]:
class Simulation(Thread):
    def __init__(self, sim_id, user=None, p_random=0.1):
        super().__init__(daemon=True)
        self.sim_id = sim_id
        self.user = user
        if self.user is None:
            self.user = User()
        self.killed = False
        self.p_random = p_random
        self.lock = BypassableLock()
    
    def kill(self, blocking=True):
        with self.lock(blocking):
            self.killed = True
    
    def run(self):
        while not self.killed:
            with self.lock:
                ad = self.user.select_ad(p_random=self.p_random)
                did_click = self.user.interact_with(*ad)
                if not self.killed:
                    q.put((self.sim_id, did_click))
                    self.user.remember(*ad, did_click)
                    self.user.learn(p_random=self.p_random)

q = Queue()

sims = []
clicks = []
dfs = []

rolling_window_size = 100

num_users = 10
synthetic_user_ftrs = users.generate_synthetic_pca_ftrs(num_users, user_pca_normal_params)

p_randoms = [
    0.01,
    0.05,
    0.10,
    1.00
]

ps = sorted(p_randoms)
colors = {}
if len(ps) > 1:
    for i, v in enumerate(ps):
        colors[v] = i/(len(ps)-1)
elif len(ps) == 1:
    colors[ps[0]] = 1.

display(HTML("<h1>Running first iterations...</h1>"), display_id="graph")

i = 0
for p_random in p_randoms:
    for j in range(num_users):
        sims.append(Simulation(i, user=User(synthetic_user_ftrs[j]), p_random=p_random))
        clicks.append([])
        dfs.append(pd.Series(dtype="float32"))
        i += 1

try:
    for s in sims:
        s.start()

    while True:
        sim_id, did_click = q.get()
        clicks[sim_id].append(did_click)
        if len(clicks[sim_id]) <= rolling_window_size:
            dfs[sim_id] = pd.Series(clicks[sim_id]).expanding().mean()
        else:
            dfs[sim_id] = pd.Series(clicks[sim_id]).rolling(rolling_window_size).mean()
        
        fig, ax = plt.subplots(figsize=(20, 10))
        fig.patch.set_facecolor("white")

        all_ys = np.array([])
        mean_ys = []

        by_p = {}
        for s in sims:
            p_random = round(s.p_random, 5)
            if p_random not in by_p:
                by_p[p_random] = []
            by_p[p_random].append(dfs[s.sim_id].values)

            ax.plot(dfs[s.sim_id], alpha=0.5, linestyle="dashed", color=cm.rainbow(colors[p_random]))
            all_ys = np.concatenate([all_ys, dfs[s.sim_id].values])
        
        for p_random in sorted(by_p.keys()):
            if len(by_p[p_random]) == 0:
                m = np.array([])
            else:
                max_len = max(len(x) for x in by_p[p_random])
                for i, x in enumerate(by_p[p_random]):
                    y = np.empty(max_len)
                    y[:len(x)] = x
                    y[len(x):] = np.nan
                    by_p[p_random][i] = y
                m = np.mean(np.stack(by_p[p_random]), axis=0)
            ax.plot(m, linewidth=2, color=cm.rainbow(colors[p_random]), label=f"p={p_random}")
            if len(m[~np.isnan(m)]):
                mean_ys.append(m[~np.isnan(m)][-1])
        
        ax.set_xlabel("Iterations")
        ax.set_ylabel("CTR")
        ax.set_title(f"Average CTR over last {rolling_window_size} iterations.")
        # ax.set_title("Average CTR over iterations.")
        ax.legend()

        ax.set_ylim(top=1.1*max([np.percentile(all_ys[~np.isnan(all_ys)], 90), *mean_ys]))
 
        update_display(fig, display_id="graph")
        plt.close()

except KeyboardInterrupt:
    pass
finally:
    plt.close()
    for s in sims:
        s.kill(False)